Downloading this data: https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires/data

### Load libraries

In [1]:
import os
import shutil

#Path where I saved kaggle.json 
source_path = '/Users/jonzimmerman/Desktop/Data Projects/Wildfires/data/kaggle.json' 

### Read in Kaggle token

In [2]:
#Destination path for kaggle.json
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)
shutil.copy(source_path, os.path.join(kaggle_dir, "kaggle.json"))

'/Users/jonzimmerman/.kaggle/kaggle.json'

In [3]:
#Set permission
os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

In [4]:
#!pip install kaggle

### Download SQL dataset

In [5]:
#!kaggle datasets download -d rtatman/188-million-us-wildfires

In [6]:
import zipfile

with zipfile.ZipFile("188-million-us-wildfires.zip", "r") as zip_ref:
    for file in zip_ref.namelist():
        if file.endswith(".sqlite"):
            zip_ref.extract(file, "wildfire_sqlite")
            print(f"Extracted {file} to wildfire_sqlite/")

Extracted FPA_FOD_20170508.sqlite to wildfire_sqlite/


### Just get the table you want

In [7]:
import sqlite3
import pandas as pd

db_path = "wildfire_sqlite/FPA_FOD_20170508.sqlite"
conn = sqlite3.connect(db_path)

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

#Make the pandas dataframe
df = pd.read_sql("SELECT * FROM Fires;", conn)

conn.close()

In [8]:
df.head(5)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [9]:
df.shape

(1880465, 39)